In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    fletcher=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.6167407631874084
epoch:  1, loss: 0.03265359252691269
epoch:  2, loss: 0.021853359416127205
epoch:  3, loss: 0.014389431104063988
epoch:  4, loss: 0.009962748736143112
epoch:  5, loss: 0.00967622920870781
epoch:  6, loss: 0.008795213885605335
epoch:  7, loss: 0.008696391247212887
epoch:  8, loss: 0.008534779772162437
epoch:  9, loss: 0.008449076674878597
epoch:  10, loss: 0.008374212309718132
epoch:  11, loss: 0.008369117975234985
epoch:  12, loss: 0.00836736336350441
epoch:  13, loss: 0.0080410810187459
epoch:  14, loss: 0.007960304617881775
epoch:  15, loss: 0.007912663742899895
epoch:  16, loss: 0.0077943624928593636
epoch:  17, loss: 0.007705420255661011
epoch:  18, loss: 0.007455704268068075
epoch:  19, loss: 0.007395199034363031
epoch:  20, loss: 0.007271638140082359
epoch:  21, loss: 0.006751551758497953
epoch:  22, loss: 0.005550604313611984
epoch:  23, loss: 0.005236185621470213
epoch:  24, loss: 0.004805380012840033
epoch:  25, loss: 0.004683044273406267
ep

In [6]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9796319335137847
Test metrics:  R2 = 0.973907384360492


In [9]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    fletcher=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
    solver="pinv"
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].cpu().detach().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.1307525336742401
epoch:  1, loss: 0.073084257543087
epoch:  2, loss: 0.06910500675439835
epoch:  3, loss: 0.024820100516080856
epoch:  4, loss: 0.016336336731910706
epoch:  5, loss: 0.012859340757131577
epoch:  6, loss: 0.011659717187285423
epoch:  7, loss: 0.010847141966223717
epoch:  8, loss: 0.010246441699564457
epoch:  9, loss: 0.009714744053781033
epoch:  10, loss: 0.009232643991708755
epoch:  11, loss: 0.008823040872812271
epoch:  12, loss: 0.008493347093462944
epoch:  13, loss: 0.008206170983612537
epoch:  14, loss: 0.007954934611916542
epoch:  15, loss: 0.007663401309400797
epoch:  16, loss: 0.007268233690410852
epoch:  17, loss: 0.006912610959261656
epoch:  18, loss: 0.006687356159090996
epoch:  19, loss: 0.006480179261416197
epoch:  20, loss: 0.006327059119939804
epoch:  21, loss: 0.006123474333435297
epoch:  22, loss: 0.005564858205616474
epoch:  23, loss: 0.005481732543557882
epoch:  24, loss: 0.005403767805546522
epoch:  25, loss: 0.0053127543069422245
e

In [10]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.987889966202478
Test metrics:  R2 = 0.9794473056414267
